In [0]:
# Steps to connect with Kafka

# Create resource group
#   Login to azure portal
#   Click on resource groups -> Click create 
#   		resource group name: "loony_hdinsight_rg"
#   		region : East US


# Now go to the resource group
# Now we have to create virtual network
# Click on create -> search 'virtual network'
# Click on create ->
# Choose our resource group
# 		name : loony_hdinsight_kafka_vnet
#       region : East US
# Click on create

# Now go back to the resource group
# Now we can see the virtual network is there

# At this point we have to create azure hdinsights
# So for that click on create from resource groups
# search 'Azure HDInsight'
# Click on create -> 
# 		cluster name : loony-hdinsight-kafka-cluster
# 		region : east us 2
# Then click on 'select cluster type'
# There will be different types of cluster
# I am going to choose 'Kafka' from there
# Next we have to give login user name and password
# 		username : loony_admin
# 		password : Password@123
# 		ssh user : sshuser (same password)

# Click on next -> It will lead to storage
# 		Storage-account name : loonyhdinsightstorage

# Click on next -> It will lead to 'Security + networking'
# choose our virtual network from here

# Click on next -> It will be go to 'Configuration +  pricing'
# We have to choose the cheapest from all nodes
#       Head node: Change to the one that costs 21.25 INR/hour
#       Zookeeper node: Change to the one that costs 4.32 INR/hour
# 		All we have to change standard disks : 1
# 		Worker node : 3
#       Zookeeper node: Change to the one that costs 4.32 INR/hour

# Click on next : Click Review + Create
# It will take 15-20 min to create the cluster

# Configuring kafka for IP address

# Go to our resources -> click on the cluster(loony-hdinsight-kafka-cluster)
# Then click on the url https://loony-hdinsight-kafka-cluster.azurehdinsight.net
# It will ask for the username and password
# 		username : loony_admin
# 		password : Password@123
# Ths will lead to Ambari and we are going to use the kafka from here.

# On the page click on the Config History tab to show what has been set up

# Next click on kafka from left side
# Click on config and search - 'kafka-env ' in the filter
# Go to end of the code and paste following code in that

# 	IP_ADDRESS=$(hostname -i)
#     echo advertised.listeners=$IP_ADDRESS
#     sed -i.bak -e '/advertised/{/advertised@/!d;}' /usr/hdp/current/kafka-broker/conf/server.properties
#     echo "advertised.listeners=PLAINTEXT://$IP_ADDRESS:9092" >> /usr/hdp/current/kafka-broker/conf/server.properties

# Search for "listener" in the Filter box
# Set to:

# PLAINTEXT://0.0.0.0:9092 - to access all ports

# Then click on save -> updates

# Next go to the drop down of the 'Actions' on top right
# Choose 'Turn on maintenance mode' -> Click Okay

# Next go and click on 'Restart' -> Restart all affected 
# Now we will be able to seeall the background operations are restarting

# Once everything has started -> Click on Okay
# Now go to summary we can all the components has started

# IMPORTANT Please click through and copy the new values for this, they will not always be the same

# Kafka : IP Address: 10.1.0.7
# 	Hostname: wn0-loony.u2wcvf4dyteenp3u0ebclefmcd.bx.internal.cloudapp.net
# Kafka : IP Address: 10.1.0.5
# 	Hostname: wn1-loony.u2wcvf4dyteenp3u0ebclefmcd.bx.internal.cloudapp.net
# Kafka : IP Address: 10.1.0.6
# 	Hostname: wn2-loony.u2wcvf4dyteenp3u0ebclefmcd.bx.internal.cloudapp.net


# ZooKeeper Server : IP Address: 10.1.0.11
# ZooKeeper Server : IP Address: 10.1.0.12
# ZooKeeper Server : IP Address: 10.1.0.13


# Creating Kafka topic

# Go to resource group -> Click on hdinsight cluster -> Click on ssh + cluster login 
# Choose the host name from host name
# And copy the ssh command. This is needed for the connection

# 	ssh sshuser@loony-hdinsight-kafka-cluster-ssh.azurehdinsight.net

# Next click on the cloud shell icon

# Click on the Full Screen icon on the top right oc Cloud Shell so you have Cloud Shell fill almost the entire page

# Paste the ssh command there in the command shell - ssh sshuser@loony-hdinsight-kafka-cluster-ssh.azurehdinsight.net
# Enter the password : Password@123

# Now we have connected to the hdinsight kafka cluster

# export TOPIC="advertising"

# export ZOOKEEPERS="10.1.0.11,10.1.0.12,10.1.0.13"

# /usr/hdp/current/kafka-broker/bin/kafka-topics.sh \
# --create --replication-factor 3 --partitions 4 --topic $TOPIC --zookeeper $ZOOKEEPERS

# run the producer using the below command:
# export BROKERS="wn0-loony.u2wcvf4dyteenp3u0ebclefmcd.bx.internal.cloudapp.net:9092,wn1-loony.u2wcvf4dyteenp3u0ebclefmcd.bx.internal.cloudapp.net:9092,wn2-loony.u2wcvf4dyteenp3u0ebclefmcd.bx.internal.cloudapp.net:9092"

# /usr/hdp/current/kafka-broker/bin/kafka-console-producer.sh \
# --broker-list $BROKERS --topic $TOPIC

# Go to loony_databricks_rg and click on loony_databricks_workspace
# Under Setting, click on Virtual Network Peerings > Add Peering
# Give the following details:

# 	Name: loony_databricks_peer
# 	Virtual Network : loony_hdinsight_kafka_vnet (choose this one)
# 	Tick on Allow forwarded traffic

# 	Copy the Databricks Virtual Network Resource ID : /subscriptions/50ee833b-7a1c-449f-ad1f-ab670f962831/resourceGroups/databricks-rg-loony_databricks_workspace-23xmbnbhpg73u/providers/Microsoft.Network/virtualNetworks/workers-vnet

# 	Click on Add

# Now the status will be Initiated

# Go to Vnet> Peerings
# Click on +Add
# Give the following details

#     This virtual network
# 	Peering link name: loony_databricks_vnet_peer

# 	Remote virtual network
# 	Peering link name: loony_remote_databricks_vnet_peer

# 	Click on I know my resource ID and paste the previously copied resource id 
# 	Create

# Go to loony_databricks_workspace > Virtual Network Peerings now we see the status is connected
# Launch the workspace and create a new notebook


In [0]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import current_timestamp, window
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import *

In [0]:
schema = StructType([StructField("Daily Time Spent on Site", FloatType(), True),
                     StructField("Age", IntegerType(), True),
                     StructField("Area Income", FloatType(), True),
                     StructField("Daily Internet Usage", FloatType(), True),
                     StructField("Male", IntegerType(), True),
                     StructField("Timestamp", TimestampType(), True),
                     StructField("Clicked on Ad", IntegerType(), True)
                    ])

###### TODO Recording

- Batch 1 from the advertising.json file

{ "Daily Time Spent on Site": 80.67, "Age": 34, "Area Income": 58909.36, "Daily Internet Usage": 239.76, "Male": 0, "Timestamp": "2016-01-01 2:52:10", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 68.01, "Age": 25, "Area Income": 68357.96, "Daily Internet Usage": 188.32, "Male": 1, "Timestamp": "2016-01-01 3:35:35", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 80.94, "Age": 36, "Area Income": 60803, "Daily Internet Usage": 239.94, "Male": 0, "Timestamp": "2016-01-01 5:31:22", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 78.77, "Age": 28, "Area Income": 63497.62, "Daily Internet Usage": 211.83, "Male": 0, "Timestamp": "2016-01-01 8:27:06", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 36.56, "Age": 29, "Area Income": 42838.29, "Daily Internet Usage": 195.89, "Male": 0, "Timestamp": "2016-01-01 15:14:24", "Clicked on Ad": 1}

###### TODO Recording

- Expand the data frame fields and show the fields

In [0]:
advertising_df = spark.readStream \
                      .format("kafka") \
                      .option("kafka.bootstrap.servers", "10.1.0.5:9092, 10.1.0.6:9092, 10.1.0.7:9092") \
                      .option("subscribe", "advertising") \
                      .option("startingOffsets", "earliest") \
                      .load()

In [0]:
# After reading the data I am casting all to string, Now we will be able to access the data

In [0]:
advertising_cast_df = advertising_df.selectExpr("CAST(value AS STRING)")

advertising_cast_df.display()

value
"{ ""Daily Time Spent on Site"": 80.67, ""Age"": 34, ""Area Income"": 58909.36, ""Daily Internet Usage"": 239.76, ""Male"": 0, ""Timestamp"": ""2016-01-01 2:52:10"", ""Clicked on Ad"": 0},"
"{ ""Daily Time Spent on Site"": 36.56, ""Age"": 29, ""Area Income"": 42838.29, ""Daily Internet Usage"": 195.89, ""Male"": 0, ""Timestamp"": ""2016-01-01 15:14:24"", ""Clicked on Ad"": 1}"
"{ ""Daily Time Spent on Site"": 68.01, ""Age"": 25, ""Area Income"": 68357.96, ""Daily Internet Usage"": 188.32, ""Male"": 1, ""Timestamp"": ""2016-01-01 3:35:35"", ""Clicked on Ad"": 0},"
"{ ""Daily Time Spent on Site"": 80.94, ""Age"": 36, ""Area Income"": 60803, ""Daily Internet Usage"": 239.94, ""Male"": 0, ""Timestamp"": ""2016-01-01 5:31:22"", ""Clicked on Ad"": 0},"
"{ ""Daily Time Spent on Site"": 78.77, ""Age"": 28, ""Area Income"": 63497.62, ""Daily Internet Usage"": 211.83, ""Male"": 0, ""Timestamp"": ""2016-01-01 8:27:06"", ""Clicked on Ad"": 0},"
"{ ""Daily Time Spent on Site"": 81.21, ""Age"": 36, ""Area Income"": 63394.41, ""Daily Internet Usage"": 233.04, ""Male"": 0, ""Timestamp"": ""2016-01-02 9:30:11"", ""Clicked on Ad"": 0},"
"{ ""Daily Time Spent on Site"": 46.61, ""Age"": 42, ""Area Income"": 65856.74, ""Daily Internet Usage"": 136.18, ""Male"": 0, ""Timestamp"": ""2016-01-01 20:17:49"", ""Clicked on Ad"": 1},"
"{ ""Daily Time Spent on Site"": 75.32, ""Age"": 28, ""Area Income"": 59998.5, ""Daily Internet Usage"": 233.6, ""Male"": 1, ""Timestamp"": ""2016-01-01 21:58:55"", ""Clicked on Ad"": 0},"
"{ ""Daily Time Spent on Site"": 79.36, ""Age"": 33, ""Area Income"": 62330.75, ""Daily Internet Usage"": 234.72, ""Male"": 1, ""Timestamp"": ""2016-01-02 4:50:44"", ""Clicked on Ad"": 0},"
"{ ""Daily Time Spent on Site"": 35.25, ""Age"": 50, ""Area Income"": 47051.02, ""Daily Internet Usage"": 194.44, ""Male"": 0, ""Timestamp"": ""2016-01-03 4:39:47"", ""Clicked on Ad"": 1},"


In [0]:
# Next convert the string to schema

In [0]:
advertising_data = advertising_cast_df.select(from_json(col("value").cast("string"), schema)) \
                          .withColumnRenamed("from_json(CAST(value AS STRING))", "data")

advertising_data.display()

data
"List(80.67, 34, 58909.36, 239.76, 0, 2016-01-01T02:52:10.000+0000, 0)"
"List(36.56, 29, 42838.29, 195.89, 0, 2016-01-01T15:14:24.000+0000, 1)"
"List(68.01, 25, 68357.96, 188.32, 1, 2016-01-01T03:35:35.000+0000, 0)"
"List(80.94, 36, 60803.0, 239.94, 0, 2016-01-01T05:31:22.000+0000, 0)"
"List(78.77, 28, 63497.62, 211.83, 0, 2016-01-01T08:27:06.000+0000, 0)"
"List(81.21, 36, 63394.41, 233.04, 0, 2016-01-02T09:30:11.000+0000, 0)"
"List(46.61, 42, 65856.74, 136.18, 0, 2016-01-01T20:17:49.000+0000, 1)"
"List(75.32, 28, 59998.5, 233.6, 1, 2016-01-01T21:58:55.000+0000, 0)"
"List(79.36, 33, 62330.75, 234.72, 1, 2016-01-02T04:50:44.000+0000, 0)"
"List(35.25, 50, 47051.02, 194.44, 0, 2016-01-03T04:39:47.000+0000, 1)"


In [0]:
# Here we have splitted the data according to the column

In [0]:
advertising_data = advertising_data.select(col('data.*'))

advertising_data.display()

Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Male,Timestamp,Clicked on Ad
78.77,28,63497.62,211.83,0,2016-01-01T08:27:06.000+0000,0
81.21,36,63394.41,233.04,0,2016-01-02T09:30:11.000+0000,0
80.67,34,58909.36,239.76,0,2016-01-01T02:52:10.000+0000,0
36.56,29,42838.29,195.89,0,2016-01-01T15:14:24.000+0000,1
46.61,42,65856.74,136.18,0,2016-01-01T20:17:49.000+0000,1
68.01,25,68357.96,188.32,1,2016-01-01T03:35:35.000+0000,0
75.32,28,59998.5,233.6,1,2016-01-01T21:58:55.000+0000,0
80.94,36,60803.0,239.94,0,2016-01-01T05:31:22.000+0000,0
79.36,33,62330.75,234.72,1,2016-01-02T04:50:44.000+0000,0
35.25,50,47051.02,194.44,0,2016-01-03T04:39:47.000+0000,1


In [0]:
advertising_data.printSchema()

root
-- Daily Time Spent on Site: float (nullable = true)
-- Age: integer (nullable = true)
-- Area Income: float (nullable = true)
-- Daily Internet Usage: float (nullable = true)
-- Male: integer (nullable = true)
-- Timestamp: timestamp (nullable = true)
-- Clicked on Ad: integer (nullable = true)

##### Tumbling Window

###### TODO Recording

- Run the cell below
- There will be exactly one window with the minimum usage for that day
- Expand the knob at the bottom right of the results pane
- Expand the start and end times and show the result

In [0]:
min_use = advertising_data.groupBy(window("Timestamp", "1 days")) \
                          .agg({"Daily Internet Usage": "min"})

display(min_use)

window,min(Daily Internet Usage)
"List(2016-01-05T00:00:00.000+0000, 2016-01-06T00:00:00.000+0000)",119.03
"List(2016-01-01T00:00:00.000+0000, 2016-01-02T00:00:00.000+0000)",136.18
"List(2016-01-08T00:00:00.000+0000, 2016-01-09T00:00:00.000+0000)",126.29
"List(2016-01-04T00:00:00.000+0000, 2016-01-05T00:00:00.000+0000)",128.37
"List(2016-01-02T00:00:00.000+0000, 2016-01-03T00:00:00.000+0000)",132.71
"List(2016-01-03T00:00:00.000+0000, 2016-01-04T00:00:00.000+0000)",194.44
"List(2016-01-06T00:00:00.000+0000, 2016-01-07T00:00:00.000+0000)",201.15
"List(2016-01-07T00:00:00.000+0000, 2016-01-08T00:00:00.000+0000)",124.85


###### TODO Recording

- Run the cell below
- There will be 3 windows and aggregation for each
- Expand the knob at the bottom right of the results pane

In [0]:
max_use_count = advertising_data.groupBy(window("Timestamp", "6 hours")) \
                                .agg({"Daily Internet Usage": "max", "Clicked on Ad": "count"})

display(max_use_count)

window,count(Clicked on Ad),max(Daily Internet Usage)
"List(2016-01-04T06:00:00.000+0000, 2016-01-04T12:00:00.000+0000)",2,233.61
"List(2016-01-05T00:00:00.000+0000, 2016-01-05T06:00:00.000+0000)",1,144.27
"List(2016-01-04T00:00:00.000+0000, 2016-01-04T06:00:00.000+0000)",2,194.95
"List(2016-01-05T06:00:00.000+0000, 2016-01-05T12:00:00.000+0000)",3,224.2
"List(2016-01-01T00:00:00.000+0000, 2016-01-01T06:00:00.000+0000)",3,239.94
"List(2016-01-03T18:00:00.000+0000, 2016-01-04T00:00:00.000+0000)",1,249.54
"List(2016-01-08T06:00:00.000+0000, 2016-01-08T12:00:00.000+0000)",1,219.98
"List(2016-01-01T18:00:00.000+0000, 2016-01-02T00:00:00.000+0000)",2,233.6
"List(2016-01-02T00:00:00.000+0000, 2016-01-02T06:00:00.000+0000)",1,234.72
"List(2016-01-04T18:00:00.000+0000, 2016-01-05T00:00:00.000+0000)",1,222.87


##### TODO Recording

- Run the cell below
- Choose PlotOptions
- Keys: window, Values: avg(Area Income)
- Only one bar will be visible
- Add more data to the topic (Batch 2 from advertising.json)

{ "Daily Time Spent on Site": 46.61, "Age": 42, "Area Income": 65856.74, "Daily Internet Usage": 136.18, "Male": 0, "Timestamp": "2016-01-01 20:17:49", "Clicked on Ad": 1},
{ "Daily Time Spent on Site": 75.32, "Age": 28, "Area Income": 59998.5, "Daily Internet Usage": 233.6, "Male": 1, "Timestamp": "2016-01-01 21:58:55", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 79.36, "Age": 33, "Area Income": 62330.75, "Daily Internet Usage": 234.72, "Male": 1, "Timestamp": "2016-01-02 4:50:44", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 81.21, "Age": 36, "Area Income": 63394.41, "Daily Internet Usage": 233.04, "Male": 0, "Timestamp": "2016-01-02 9:30:11", "Clicked on Ad": 0}

-
- Wait for a bit, you should now see two bars

In [0]:
avg_income = advertising_data.groupBy(window("Timestamp", "1 days")) \
                             .agg({"Area Income": "avg"})

display(avg_income)

window,avg(Area Income)
"List(2016-01-05T00:00:00.000+0000, 2016-01-06T00:00:00.000+0000)",54357.05322265625
"List(2016-01-01T00:00:00.000+0000, 2016-01-02T00:00:00.000+0000)",60037.353236607145
"List(2016-01-08T00:00:00.000+0000, 2016-01-09T00:00:00.000+0000)",53588.34375
"List(2016-01-04T00:00:00.000+0000, 2016-01-05T00:00:00.000+0000)",62683.21328125
"List(2016-01-02T00:00:00.000+0000, 2016-01-03T00:00:00.000+0000)",62188.1904296875
"List(2016-01-03T00:00:00.000+0000, 2016-01-04T00:00:00.000+0000)",50780.68046875
"List(2016-01-06T00:00:00.000+0000, 2016-01-07T00:00:00.000+0000)",55433.35546875
"List(2016-01-07T00:00:00.000+0000, 2016-01-08T00:00:00.000+0000)",57565.53125


###### TODO Recording

- Run the cell below
- The result will initially have records
- Add more data to Kafka (Batch 3 from advertising.json)

{ "Daily Time Spent on Site": 44.96, "Age": 50, "Area Income": 52802, "Daily Internet Usage": 132.71, "Male": 1, "Timestamp": "2016-01-02 12:25:36", "Clicked on Ad": 1},
{ "Daily Time Spent on Site": 83.26, "Age": 40, "Area Income": 70225.6, "Daily Internet Usage": 187.76, "Male": 1, "Timestamp": "2016-01-02 14:36:03", "Clicked on Ad": 0}
{ "Daily Time Spent on Site": 35.25, "Age": 50, "Area Income": 47051.02, "Daily Internet Usage": 194.44, "Male": 0, "Timestamp": "2016-01-03 4:39:47", "Clicked on Ad": 1},
{ "Daily Time Spent on Site": 65.07, "Age": 24, "Area Income": 57545.56, "Daily Internet Usage": 233.85, "Male": 0, "Timestamp": "2016-01-03 7:13:53", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 76.58, "Age": 46, "Area Income": 41884.64, "Daily Internet Usage": 258.26, "Male": 0, "Timestamp": "2016-01-03 16:30:51", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 88.72, "Age": 32, "Area Income": 43870.51, "Daily Internet Usage": 211.87, "Male": 1, "Timestamp": "2016-01-03 17:10:05", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 72.8, "Age": 35, "Area Income": 63551.67, "Daily Internet Usage": 249.54, "Male": 0, "Timestamp": "2016-01-03 23:21:26", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 48.86, "Age": 35, "Area Income": 62463.7, "Daily Internet Usage": 128.37, "Male": 1, "Timestamp": "2016-01-04 0:44:57", "Clicked on Ad": 1}

-
- There will now be a total of 7 records
- Expand the results using knob on the bottom right

In [0]:
avg_income_clicked = advertising_data.groupBy(window("Timestamp", "1 days"), advertising_data["Clicked on Ad"]) \
                                     .agg({"Daily Internet Usage": "avg"}) \
                                     .withColumnRenamed("avg(Daily Internet Usage)", "avg_usage") \
                                     .select("window.start", "window.end", "Clicked on Ad", "avg_usage") \
                                     .orderBy("window.start")

display(avg_income_clicked)

start,end,Clicked on Ad,avg_usage
2016-01-01T00:00:00.000+0000,2016-01-02T00:00:00.000+0000,1,166.03499603271484
2016-01-01T00:00:00.000+0000,2016-01-02T00:00:00.000+0000,0,222.69000244140625
2016-01-02T00:00:00.000+0000,2016-01-03T00:00:00.000+0000,1,132.7100067138672
2016-01-02T00:00:00.000+0000,2016-01-03T00:00:00.000+0000,0,218.50666300455728
2016-01-03T00:00:00.000+0000,2016-01-04T00:00:00.000+0000,0,238.38000106811523
2016-01-03T00:00:00.000+0000,2016-01-04T00:00:00.000+0000,1,194.44000244140625
2016-01-04T00:00:00.000+0000,2016-01-05T00:00:00.000+0000,0,214.91749954223633
2016-01-04T00:00:00.000+0000,2016-01-05T00:00:00.000+0000,1,128.3699951171875
2016-01-05T00:00:00.000+0000,2016-01-06T00:00:00.000+0000,0,207.70999654134116
2016-01-05T00:00:00.000+0000,2016-01-06T00:00:00.000+0000,1,166.7300018310547


##### Sliding Window

###### TODO Recording

- Run the code below
- There should be 7 rows in the result
- Add more data to Kafka (Batch 4 from advertising.json)

{ "Daily Time Spent on Site": 83.16, "Age": 41, "Area Income": 70185.06, "Daily Internet Usage": 194.95, "Male": 0, "Timestamp": "2016-01-04 4:00:35", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 73.88, "Age": 29, "Area Income": 63109.74, "Daily Internet Usage": 233.61, "Male": 0, "Timestamp": "2016-01-04 6:37:15","Clicked on Ad": 0}
{ "Daily Time Spent on Site": 66.89, "Age": 23, "Area Income": 64433.99, "Daily Internet Usage": 208.24, "Male": 1, "Timestamp": "2016-01-04 7:28:43", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 83.91,"Age": 29, "Area Income": 53223.58, "Daily Internet Usage": 222.87, "Male": 0, "Timestamp": "2016-01-04 22:27:25", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 46.37, "Age": 52, "Area Income": 32847.53, "Daily Internet Usage": 144.27, "Male": 0, "Timestamp": "2016-01-05 0:02:53", "Clicked on Ad": 1}
{ "Daily Time Spent on Site": 76.44, "Age": 26, "Area Income": 58820.16, "Daily Internet Usage": 224.2, "Male": 1, "Timestamp": "2016-01-05 6:34:20", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 61.22, "Age": 45, "Area Income": 63883.81, "Daily Internet Usage": 119.03, "Male": 1, "Timestamp": "2016-01-05 9:42:22", "Clicked on Ad": 1},
{ "Daily Time Spent on Site": 71.23, "Age": 52, "Area Income": 41521.28, "Daily Internet Usage": 122.59, "Male": 0, "Timestamp": "2016-01-05 11:53:17", "Clicked on Ad": 1},
{ "Daily Time Spent on Site": 65.56, "Age": 25, "Area Income": 69646.35, "Daily Internet Usage": 181.25, "Male": 1, "Timestamp": "2016-01-05 12:59:07", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 45.11, "Age": 58, "Area Income": 39799.73, "Daily Internet Usage": 195.69, "Male": 0, "Timestamp": "2016-01-10 16:26:44", "Clicked on Ad": 1},
{ "Daily Time Spent on Site": 79.54, "Age": 44, "Area Income": 70492.6, "Daily Internet Usage": 217.68, "Male": 1, "Timestamp": "2016-01-10 16:34:31", "Clicked on Ad": 0}


-
- There should be more results in the output

In [0]:
daily_time_avg = advertising_data.groupBy(window("Timestamp", "5 days", "2 days"), "Clicked on Ad") \
                                   .agg({"Daily Time Spent on Site": "avg"}) \
                                   .orderBy("window.start") 

display(daily_time_avg)

window,Clicked on Ad,avg(Daily Time Spent on Site)
"List(2015-12-29T00:00:00.000+0000, 2016-01-03T00:00:00.000+0000)",0,78.44250011444092
"List(2015-12-29T00:00:00.000+0000, 2016-01-03T00:00:00.000+0000)",1,42.710000356038414
"List(2015-12-31T00:00:00.000+0000, 2016-01-05T00:00:00.000+0000)",1,42.44800033569336
"List(2015-12-31T00:00:00.000+0000, 2016-01-05T00:00:00.000+0000)",0,77.40937566757202
"List(2016-01-02T00:00:00.000+0000, 2016-01-07T00:00:00.000+0000)",1,57.28888914320204
"List(2016-01-02T00:00:00.000+0000, 2016-01-07T00:00:00.000+0000)",0,77.1606674194336
"List(2016-01-04T00:00:00.000+0000, 2016-01-09T00:00:00.000+0000)",1,62.310909964821555
"List(2016-01-04T00:00:00.000+0000, 2016-01-09T00:00:00.000+0000)",0,76.38636432994495
"List(2016-01-06T00:00:00.000+0000, 2016-01-11T00:00:00.000+0000)",0,77.71750068664551
"List(2016-01-06T00:00:00.000+0000, 2016-01-11T00:00:00.000+0000)",1,66.86800079345703


###### TODO Recording

- Run the cell below
- There should be 18 rows in the result
- Add data from Batch 5 in advertising.json

{ "Daily Time Spent on Site": 78.29, "Age": 38, "Area Income": 57844.96, "Daily Internet Usage": 252.07, "Male": 0, "Timestamp": "2016-01-05 20:58:42", "Clicked on Ad": 1},
{ "Daily Time Spent on Site": 81.03, "Age": 28, "Area Income": 63727.5, "Daily Internet Usage": 201.15, "Male": 0, "Timestamp": "2016-01-06 13:20:01", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 84.31, "Age": 29, "Area Income": 47139.21, "Daily Internet Usage": 225.87, "Male": 0, "Timestamp": "2016-01-06 21:43:22", "Clicked on Ad": 1},
{ "Daily Time Spent on Site": 81.22, "Age": 53, "Area Income": 34309.24, "Daily Internet Usage": 223.09, "Male": 1, "Timestamp": "2016-01-07 13:25:21", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 51.56, "Age": 46, "Area Income": 63102.19, "Daily Internet Usage": 124.85, "Male": 0, "Timestamp": "2016-01-07 13:58:51", "Clicked on Ad": 1},
{ "Daily Time Spent on Site": 68.11, "Age": 38, "Area Income": 73884.48, "Daily Internet Usage": 231.21, "Male": 0, "Timestamp": "2016-01-07 21:21:50", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 66.08, "Age": 61, "Area Income": 58966.22, "Daily Internet Usage": 184.23, "Male": 1, "Timestamp": "2016-01-07 23:02:43", "Clicked on Ad": 1},
{ "Daily Time Spent on Site": 80.51, "Age": 28, "Area Income": 64008.55, "Daily Internet Usage": 200.28, "Male": 1, "Timestamp": "2016-01-08 2:34:06", "Clicked on Ad": 0},
{ "Daily Time Spent on Site": 78.76, "Age": 24, "Area Income": 46422.76, "Daily Internet Usage": 219.98, "Male": 1, "Timestamp": "2016-01-08 8:08:47", "Clicked on Ad": 1},
{ "Daily Time Spent on Site": 53.63, "Age": 54, "Area Income": 50333.72, "Daily Internet Usage": 126.29, "Male": 1, "Timestamp": "2016-01-08 18:13:43", "Clicked on Ad": 1},
{ "Daily Time Spent on Site": 79.97, "Age": 26, "Area Income": 61747.98, "Daily Internet Usage": 185.45, "Male": 1, "Timestamp": "2016-01-08 19:38:45", "Clicked on Ad": 0}


-
- Show 24 rows in the result

In [0]:
clicked_count = advertising_data.groupBy(window("Timestamp", "5 days", "1 day"), "Clicked on Ad") \
                                .count() \
                                .orderBy("window.start", "Clicked on Ad")

display(clicked_count)

window,Clicked on Ad,count
"List(2015-12-28T00:00:00.000+0000, 2016-01-02T00:00:00.000+0000)",0,5
"List(2015-12-28T00:00:00.000+0000, 2016-01-02T00:00:00.000+0000)",1,2
"List(2015-12-29T00:00:00.000+0000, 2016-01-03T00:00:00.000+0000)",0,8
"List(2015-12-29T00:00:00.000+0000, 2016-01-03T00:00:00.000+0000)",1,3
"List(2015-12-30T00:00:00.000+0000, 2016-01-04T00:00:00.000+0000)",0,12
"List(2015-12-30T00:00:00.000+0000, 2016-01-04T00:00:00.000+0000)",1,4
"List(2015-12-31T00:00:00.000+0000, 2016-01-05T00:00:00.000+0000)",0,16
"List(2015-12-31T00:00:00.000+0000, 2016-01-05T00:00:00.000+0000)",1,5
"List(2016-01-01T00:00:00.000+0000, 2016-01-06T00:00:00.000+0000)",0,19
"List(2016-01-01T00:00:00.000+0000, 2016-01-06T00:00:00.000+0000)",1,10
